In [1]:
import os
import sqlite3
from datetime import datetime
from tqdm import tqdm_notebook

import pandas as pd
import numpy as np
import geopandas as gpd

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import plotly_express as px
import plotly.graph_objects as go

from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import statsmodels.tsa.api as smt
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose


def f_requete_sql (requete) :
    try:
        connexion = sqlite3.connect('./data/liste_piezos.db')
        curseur = connexion.cursor()
        print("Connexion réussie à SQLite")
        curseur.execute(requete)
        connexion.commit()
        resultat = curseur.fetchall()
        curseur.close()
        connexion.close()
        print("Connexion SQLite est fermée")
        return resultat
    except sqlite3.Error as error:
        print("Erreur lors du mis à jour dans la table", error)
        
        
def cmyk_to_rgb(c, m, y, k, cmyk_scale, rgb_scale=255):
    r = rgb_scale * (1.0 - c / float(cmyk_scale)) * (1.0 - k / float(cmyk_scale))
    g = rgb_scale * (1.0 - m / float(cmyk_scale)) * (1.0 - k / float(cmyk_scale))
    b = rgb_scale * (1.0 - y / float(cmyk_scale)) * (1.0 - k / float(cmyk_scale))
    
    if rgb_scale == 255 :      
        return list(map(int, r)), list(map(int, g)), list(map(int, b))
    else :
        return r, g, b

# Chargement des données de clustering piezo

In [2]:
 requete = """
        SELECT code_bss, latitude, longitude, altitude_station, profondeur_investigation, codes_bdlisa, cluster_kmeans, FRANCE_lvl_1, FRANCE_lvl_2, FRANCE_lvl_3, FRANCE_lvl_4, EtatEH, NatureEH, MilieuEH, ThemeEH, OrigineEH
        FROM data_piezo
        INNER JOIN TME
        ON codes_bdlisa = CodeEH
        WHERE cluster_kmeans IS NOT NULL
        """

data_piezo = pd.DataFrame(f_requete_sql(requete))
data_piezo.columns = ["code_bss", "latitude", "longitude", "altitude_station", "profondeur_investigation", "codes_bdlisa", "cluster_kmeans", "FRANCE_lvl_1", "FRANCE_lvl_2", "FRANCE_lvl_3", "FRANCE_lvl_4", "EtatEH", "NatureEH", "MilieuEH", "ThemeEH", "OrigineEH"]
data_piezo.set_index("code_bss", inplace=True)
data_piezo.head()

Connexion réussie à SQLite
Connexion SQLite est fermée


,latitude,longitude,altitude_station,profondeur_investigation,codes_bdlisa,cluster_kmeans,FRANCE_lvl_1,FRANCE_lvl_2,FRANCE_lvl_3,FRANCE_lvl_4,EtatEH,NatureEH,MilieuEH,ThemeEH,OrigineEH
code_bss,,,,,,,,,,,,,,,
03423X0056_100,48.190195,7.396716,174.0,10.7,221AA15,GES_C,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3121,2,5,1,1,1
06216X0228_P30-10,46.272865,3.401730,237.2,8.0,932AR01,ARA_A,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3122,2,5,1,1,1
09145X0057_N210,44.110203,4.751991,31.0,14.5,712IA01,PAC_A,FRANCE_3,FRANCE_31,FRANCE_313,FRANCE_3131,2,5,1,1,1
06932X0178_P10,45.892458,2.972177,801.8,48.0,153AA01,ARA_A,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3122,2,5,4,5,1
06505X0093_PZ10,46.128984,4.761390,175.0,19.0,507AD00,ARA_A,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3122,2,7,1,2,3


In [3]:
data_piezo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1193 entries, 03423X0056_100 to 04994X0528_FPZ
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   latitude                  1193 non-null   float64
 1   longitude                 1193 non-null   float64
 2   altitude_station          1193 non-null   float64
 3   profondeur_investigation  1156 non-null   float64
 4   codes_bdlisa              1193 non-null   object 
 5   cluster_kmeans            1193 non-null   object 
 6   FRANCE_lvl_1              1193 non-null   object 
 7   FRANCE_lvl_2              1193 non-null   object 
 8   FRANCE_lvl_3              1193 non-null   object 
 9   FRANCE_lvl_4              1193 non-null   object 
 10  EtatEH                    1193 non-null   object 
 11  NatureEH                  1193 non-null   object 
 12  MilieuEH                  1193 non-null   object 
 13  ThemeEH                   1193 non-null   obj

In [4]:
dict_etat = {"1":"Entité hydrogéologique à nappe captive" , 
             "2":"Entité hydrogéologique à nappe libre", 
             "3": "Entité hydrogéologique à parties libres et captives", 
             "4":"Entité hydrogéologique alternativement libre puis captive"}
dict_nature = {"0":"inconnue",
               "3":"Système aquifère", 
               "5":"Unité aquifère", 
               "6":"Unité semi-perméable",
               "7":"Unité imperméable"}
dict_milieu = {"1":"Poreux", 
               "2":"Sédimentaire", 
               "3": "Karstique", 
               "4":"Matricielle / fissures",
               "5": "Karstique / fissures",
               "6": "Fractures et/ou fissures",
               "8":"Matricielle / karstique",
               "9":"Matrice/fracture/karst"}
dict_theme = {"1":"Alluvial", 
              "2":"Sédimentaire", 
              "3":"Matricielle / fissures" , 
              "4":"Intensément plissés de montagne"}
dict_origine = {"1":"Carte géologique ou hydrogéologique", 
                "2":"Complétude Totale", 
                "3":"Complétude Partielle", 
                "4":"Agrégation par héritage"}

In [5]:
data_piezo.replace({"EtatEH": dict_etat}, inplace=True)
data_piezo.replace({"NatureEH": dict_nature}, inplace=True) 
data_piezo.replace({"MilieuEH": dict_milieu}, inplace=True) 
data_piezo.replace({"ThemeEH": dict_theme}, inplace=True) 

In [6]:
data_piezo.head()

,latitude,longitude,altitude_station,profondeur_investigation,codes_bdlisa,cluster_kmeans,FRANCE_lvl_1,FRANCE_lvl_2,FRANCE_lvl_3,FRANCE_lvl_4,EtatEH,NatureEH,MilieuEH,ThemeEH,OrigineEH
code_bss,,,,,,,,,,,,,,,
03423X0056_100,48.190195,7.396716,174.0,10.7,221AA15,GES_C,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3121,Entité hydrogéologique à nappe libre,Unité aquifère,Poreux,Alluvial,1
06216X0228_P30-10,46.272865,3.401730,237.2,8.0,932AR01,ARA_A,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3122,Entité hydrogéologique à nappe libre,Unité aquifère,Poreux,Alluvial,1
09145X0057_N210,44.110203,4.751991,31.0,14.5,712IA01,PAC_A,FRANCE_3,FRANCE_31,FRANCE_313,FRANCE_3131,Entité hydrogéologique à nappe libre,Unité aquifère,Poreux,Alluvial,1
06932X0178_P10,45.892458,2.972177,801.8,48.0,153AA01,ARA_A,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3122,Entité hydrogéologique à nappe libre,Unité aquifère,Matricielle / fissures,5,1
06505X0093_PZ10,46.128984,4.761390,175.0,19.0,507AD00,ARA_A,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3122,Entité hydrogéologique à nappe libre,Unité imperméable,Poreux,Sédimentaire,3


# Chargement de la carte lithologique

In [7]:
URL = 'http://mapsref.brgm.fr/wxs/infoterre/catalogue?SERVICE=WFS&REQUEST=GetFeature&VERSION=2.0.0&TYPENAMES=ms:litho_1M_SIMPLIFIEE'

data_litho = gpd.read_file(URL)

data_litho[["C_FOND", "M_FOND", "J_FOND", "N_FOND"]] = data_litho[["C_FOND", "M_FOND", "J_FOND", "N_FOND"]].astype(int, copy=False)

data_litho["RGB"] = [(tuple(color)) for color in np.stack(cmyk_to_rgb(data_litho.C_FOND, data_litho.M_FOND, data_litho.J_FOND, data_litho.N_FOND, cmyk_scale = 100, rgb_scale=255)).T]

data_litho["RGB_sns"] = [(tuple(color)) for color in np.stack(cmyk_to_rgb(data_litho.C_FOND, data_litho.M_FOND, data_litho.J_FOND, data_litho.N_FOND, cmyk_scale = 100, rgb_scale=1)).T]

data_litho["RGB_str"] = [str(tuple(color)) for color in np.stack(cmyk_to_rgb(data_litho.C_FOND, data_litho.M_FOND, data_litho.J_FOND, data_litho.N_FOND, cmyk_scale = 100, rgb_scale=255)).T]

data_litho["RGB_px"] = ["rgb" + data_litho.loc[line,"RGB_str"] for line in data_litho.index]

data_litho.head()

,gml_id,OBJECTID,CODE_GEOL,SHAPE_Leng,SHAPE_Area,DESCR,TYPE,C_FOND,M_FOND,J_FOND,N_FOND,geometry,RGB,RGB_sns,RGB_str,RGB_px
0,LITHO_1M_SIMPLIFIEE.535,535,8,3042.336690,5.591750e+05,Granites,Roches Magmatiques,0,100,100,0,"POLYGON ((9.25495 41.33883, 9.24708 41.34146, ...","(255, 0, 0)","(1.0, 0.0, 0.0)","(255, 0, 0)","rgb(255, 0, 0)"
1,LITHO_1M_SIMPLIFIEE.536,536,8,5176.176558,1.301260e+06,Granites,Roches Magmatiques,0,100,100,0,"POLYGON ((9.26152 41.36311, 9.25263 41.36788, ...","(255, 0, 0)","(1.0, 0.0, 0.0)","(255, 0, 0)","rgb(255, 0, 0)"
2,LITHO_1M_SIMPLIFIEE.537,537,8,31078.613320,3.406823e+07,Granites,Roches Magmatiques,0,100,100,0,"POLYGON ((9.22235 41.43062, 9.22585 41.42359, ...","(255, 0, 0)","(1.0, 0.0, 0.0)","(255, 0, 0)","rgb(255, 0, 0)"
3,LITHO_1M_SIMPLIFIEE.538,538,6,17541.113318,1.021914e+07,Sables,Roches Sédimentaires,0,0,55,0,"POLYGON ((8.95247 41.49212, 8.94354 41.48919, ...","(255, 255, 114)","(1.0, 1.0, 0.44999999999999996)","(255, 255, 114)","rgb(255, 255, 114)"
4,LITHO_1M_SIMPLIFIEE.539,539,8,2494.234014,2.654343e+05,Granites,Roches Magmatiques,0,100,100,0,"POLYGON ((9.36211 41.54371, 9.35909 41.54462, ...","(255, 0, 0)","(1.0, 0.0, 0.0)","(255, 0, 0)","rgb(255, 0, 0)"


In [8]:
data_litho.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1364 entries, 0 to 1363
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   gml_id      1364 non-null   object  
 1   OBJECTID    1364 non-null   int64   
 2   CODE_GEOL   1364 non-null   int64   
 3   SHAPE_Leng  1364 non-null   float64 
 4   SHAPE_Area  1364 non-null   float64 
 5   DESCR       1364 non-null   object  
 6   TYPE        1364 non-null   object  
 7   C_FOND      1364 non-null   int32   
 8   M_FOND      1364 non-null   int32   
 9   J_FOND      1364 non-null   int32   
 10  N_FOND      1364 non-null   int32   
 11  geometry    1364 non-null   geometry
 12  RGB         1364 non-null   object  
 13  RGB_sns     1364 non-null   object  
 14  RGB_str     1364 non-null   object  
 15  RGB_px      1364 non-null   object  
dtypes: float64(2), geometry(1), int32(4), int64(2), object(7)
memory usage: 149.3+ KB


In [ ]:
data_litho.plot(figsize=(15,15),
          column = "DESCR",
          legend = True)

# Jointure

## Suppression des lignes NaN dans DESCR

In [9]:
data_piezo = gpd.GeoDataFrame(
    data_piezo, geometry=gpd.points_from_xy(data_piezo.longitude, data_piezo.latitude, crs="CRS84"))
data_piezo.to_crs(crs='EPSG:4326', inplace=True)

data_piezo = gpd.sjoin(data_piezo, data_litho.loc[:,["DESCR", "TYPE", "geometry", "RGB_sns"]], how='left')

data_piezo["dummy_column_for_size"] = 1

index_to_drop= data_piezo[data_piezo['DESCR'].isna()].index
data_piezo.drop(index_to_drop , inplace=True)

data_piezo.head()

,latitude,longitude,altitude_station,profondeur_investigation,codes_bdlisa,cluster_kmeans,FRANCE_lvl_1,FRANCE_lvl_2,FRANCE_lvl_3,FRANCE_lvl_4,...,NatureEH,MilieuEH,ThemeEH,OrigineEH,geometry,index_right,DESCR,TYPE,RGB_sns,dummy_column_for_size
code_bss,,,,,,,,,,,,,,,,,,,,,
03423X0056_100,48.190195,7.396716,174.0,10.7,221AA15,GES_C,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3121,...,Unité aquifère,Poreux,Alluvial,1,POINT (7.39672 48.19020),226.0,Argiles,Roches Sédimentaires,"(0.83, 1.0, 0.75)",1
06216X0228_P30-10,46.272865,3.401730,237.2,8.0,932AR01,ARA_A,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3122,...,Unité aquifère,Poreux,Alluvial,1,POINT (3.40173 46.27287),1341.0,Sables,Roches Sédimentaires,"(1.0, 1.0, 0.44999999999999996)",1
09145X0057_N210,44.110203,4.751991,31.0,14.5,712IA01,PAC_A,FRANCE_3,FRANCE_31,FRANCE_313,FRANCE_3131,...,Unité aquifère,Poreux,Alluvial,1,POINT (4.75199 44.11020),1064.0,Sables,Roches Sédimentaires,"(1.0, 1.0, 0.44999999999999996)",1
06932X0178_P10,45.892458,2.972177,801.8,48.0,153AA01,ARA_A,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3122,...,Unité aquifère,Matricielle / fissures,5,1,POINT (2.97218 45.89246),470.0,Basaltes et rhyolites,Roches Magmatiques,"(0.0, 0.30000000000000004, 0.6599999999999999)",1
06505X0093_PZ10,46.128984,4.761390,175.0,19.0,507AD00,ARA_A,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3122,...,Unité imperméable,Poreux,Sédimentaire,3,POINT (4.76139 46.12898),402.0,Argiles,Roches Sédimentaires,"(0.83, 1.0, 0.75)",1


In [10]:
data_piezo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1191 entries, 03423X0056_100 to 04994X0528_FPZ
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   latitude                  1191 non-null   float64 
 1   longitude                 1191 non-null   float64 
 2   altitude_station          1191 non-null   float64 
 3   profondeur_investigation  1154 non-null   float64 
 4   codes_bdlisa              1191 non-null   object  
 5   cluster_kmeans            1191 non-null   object  
 6   FRANCE_lvl_1              1191 non-null   object  
 7   FRANCE_lvl_2              1191 non-null   object  
 8   FRANCE_lvl_3              1191 non-null   object  
 9   FRANCE_lvl_4              1191 non-null   object  
 10  EtatEH                    1191 non-null   object  
 11  NatureEH                  1191 non-null   object  
 12  MilieuEH                  1191 non-null   object  
 13  ThemeEH               

# Graphs

## Geoplot

In [ ]:
palette = sns.color_palette("Set2")
cmap = ListedColormap(colors=palette)


fig, ax = plt.subplots(figsize=(15,15))

base = data_litho.plot(
            column = data_litho["DESCR"],
            ax = ax,  
            legend = True,
            linewidth=0.3,
            edgecolor='black',
            cmap = "Paired",
            legend_kwds={'loc': 'center left', 'bbox_to_anchor':(1,0.5)})

type = ["Argiles", "Craie"]



clusters = data_piezo.loc[data_piezo["DESCR"].isin(type)].plot(column="FRANCE_lvl_4",
            ax=ax, markersize=80, edgecolors="k", legend=False, cmap = "Accent")

plt.show()

ax = sns.countplot(x= data_piezo.loc[data_piezo["DESCR"].isin(type), "FRANCE_lvl_4"])
_= ax.set_xticklabels(ax.get_xticklabels(),rotation = 30)

## Plotly Express

### choroplet map box

In [ ]:
dict_colors = dict(zip(data_litho.DESCR.unique(), data_litho.RGB_px.unique()))

fig = px.choropleth_mapbox(data_frame = data_litho, 
                     geojson=data_litho["geometry"], 
                     locations = data_litho.index, 
                     color=data_litho["DESCR"],
                     mapbox_style='white-bg',
                     color_discrete_map=dict_colors,
                     opacity=0.5,
                     labels= {"DESCR" : "Terrain"},
                     center={"lat": 47, "lon": 2.2},
                     category_orders = {"DESCR": sorted(data_litho.DESCR.unique())},
                     zoom=4.3)


data_scat = px.scatter_mapbox(data_piezo,
                             lat="latitude",
                             lon="longitude",
                             color="FRANCE_lvl_4",
                             color_discrete_sequence = px.colors.qualitative.Light24,
                             size = "dummy_column_for_size",
                             size_max = 5,
                             hover_name = data_piezo.index,
                             hover_data={'FRANCE_lvl_4':False,
                                         'dummy_column_for_size':False,
                                         'latitude':False,
                                         'longitude':False,
                                         'DESCR': True,
                                         'TYPE': True},
                             labels={"FRANCE_lvl_4" : "Clusters"},
                              category_orders = {"FRANCE_lvl_4": sorted(data_piezo.FRANCE_lvl_4.unique())}
                             ).data
          

for item in range(len(data_scat)):
    fig.add_trace(data_scat[item])
    fig.update_traces(mode="markers", 
                      selector=dict(type='scattermapbox'))
    fig.update_layout(legend= {'itemsizing': 'constant'})

fig.show()

## Countplot

In [ ]:
data_piezo.head()

In [ ]:
# Create an array with the colors you want to use
colors = data_litho[["DESCR","RGB_sns"]].drop_duplicates().sort_values(by="DESCR")["RGB_sns"].values
# Set your custom color palette
customPalette = sns.set_palette(sns.color_palette(colors))

In [ ]:
type_terrain = "Argiles"
cluster = "FRANCE_3131"

colors = data_piezo.loc[data_piezo.FRANCE_lvl_4 == cluster,["DESCR","RGB_sns"]].drop_duplicates().sort_values(by="DESCR")["RGB_sns"].values
customPalette = sns.set_palette(sns.color_palette(colors))

In [ ]:
type_terrain = "Craie"
cluster = "FRANCE_3131"
donnees = data_piezo.loc[data_piezo.FRANCE_lvl_4 == cluster]
ax = sns.countplot(data=donnees, x="DESCR", palette=color_discrete_sequence)
ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha="right")
ax.set_xticklabels
_=plt.show()

## Analyse via crosstab

In [ ]:
donnees_cluster = pd.crosstab(data_piezo["FRANCE_lvl_3"], data_piezo["DESCR"], normalize='index')
donnees_cluster

In [ ]:
customPalette = sns.set_palette(sns.color_palette(colors))

In [ ]:
g_cluster = sns.barplot(x="Granites", y=donnees_cluster.index,
                data=donnees_cluster,
                palette = customPalette
                saturation=.5)

In [ ]:
donnees_terrain = pd.crosstab(data_piezo["DESCR"], data_piezo["FRANCE_lvl_3"],  normalize='index')
donnees_terrain

In [ ]:
# Create an array with the colors you want to use
colors = data_litho[["DESCR","RGB_sns"]].drop_duplicates().sort_values(by="DESCR")["RGB_sns"].values
# Set your custom color palette
customPalette = sns.set_palette(sns.color_palette(colors))

In [ ]:
customPalette

In [ ]:
g_terrain = sns.barplot(x="FRANCE_311", y=donnees_terrain.index,
                data=donnees_terrain,
                palette=customPalette,
                saturation=.5)

## IN_APP

In [ ]:
def f_count_plot(data_piezo, data_litho, filtre, valeur, cluster_level):
    # Create an array with the colors you want to use
    

    fig, ax = plt.subplots()
    
    if filtre == "Terrain" :
        
        donnees_cluster = pd.crosstab(data_piezo["FRANCE_lvl_3"], data_piezo["DESCR"], normalize='index')
        ax = sns.barplot(x=valeur, y=donnees_cluster.index,
                data=donnees_cluster, 
                saturation=.5)

        ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha="right")
        ax.set_ylabel('')
        ax.set_xlabel('Clusters')

    if filtre == "Cluster" :
        
        donnees_terrain = pd.crosstab(data_piezo["DESCR"], data_piezo[cluster_level],  normalize='index')
        colors = data_piezo.loc[data_piezo[cluster_level] == valeur,["DESCR","RGB_sns"]].drop_duplicates().sort_values(by="DESCR")["RGB_sns"].values
        # Set your custom color palette
        customPalette = sns.set_palette(sns.color_palette(colors))

        ax = sns.barplot(x=valeur, y=donnees_terrain.index,
                        palette = customPalette,
                        data=donnees_terrain,
                        order = sorted(donnees_terrain.index.drop_duplicates().values))
        ax.set_xlabel('')
        ax.set_ylabel('')

# Affichage Chronique

## All piezo

In [ ]:
level = "FRANCE_lvl_2"
clusters = ["21", '31', "11", "12"]
palette = px.colors.qualitative.Light24

fig = go.Figure()

for index, cluster in enumerate(clusters) :
    custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
    chroniques = pd.read_csv(f"./Clustering/data/FRANCE.csv", sep = ";", parse_dates=True, index_col="date_mesure", date_parser=custom_date_parser)

    requete = f"""
            SELECT code_bss
            FROM cluster_list
            WHERE {level} = {cluster}
            """

    list_chroniques = np.stack(pd.DataFrame(f_requete_sql(requete)).values.T)[0].tolist()

    # plot piezos
    fig_all = px.line(chroniques[list_chroniques], title='Evolution temporelle standardisée des chroniques')
    fig_all.update_traces(line=dict(width=1, color = palette[index]), opacity = 0.4)
    fig_all.update_layout(showlegend=False)
    
    # plot moyenne du cluster
    fig_mean = px.line(chroniques[list_chroniques].mean(axis=1))
    fig_mean.data[0].name = cluster
    fig_mean.update_traces(line=dict(width=2, color = palette[index]), opacity = 1)
    
    fig.add_trace(fig_mean.data[0])
    fig.add_traces(fig_all.data)
    
    fig_all.update_layout(legend_title="Chroniques")

    
for trace in range(len(fig['data'])):
    if len(fig['data'][trace]['name']) > 4:
        fig['data'][trace]['showlegend'] = False
    else :
        fig['data'][trace]['showlegend'] = True
fig

## Only mean

In [ ]:
level = "FRANCE_lvl_2"
clusters = ["21", '31', "11", "12"]


requete = f"""
        SELECT code_bss
        FROM cluster_list
        WHERE {level} = {clusters[0]}
        """
list_chroniques = np.stack(pd.DataFrame(f_requete_sql(requete)).values.T)[0].tolist()

custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
chroniques = pd.read_csv(f"./Clustering/data/FRANCE.csv", sep = ";", parse_dates=True, index_col="date_mesure", date_parser=custom_date_parser)

df_clusters = pd.DataFrame()
df_clusters[clusters[0]] = chroniques[list_chroniques].mean(axis=1)

for index, cluster in enumerate(clusters) :
    if index > 0 :
        requete = f"""
            SELECT code_bss
            FROM cluster_list
            WHERE {level} = {cluster}
            """
        list_chroniques = np.stack(pd.DataFrame(f_requete_sql(requete)).values.T)[0].tolist()

        custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
        chroniques = pd.read_csv(f"./Clustering/data/FRANCE.csv", sep = ";", parse_dates=True, index_col="date_mesure", date_parser=custom_date_parser)

        df_clusters[cluster] = chroniques[list_chroniques].mean(axis=1)

In [ ]:
px.line(df_clusters)

In [72]:
st_cluster_level = 3

dict_cluster_levels = {1: "FRANCE_lvl_1",
                    2: "FRANCE_lvl_2",
                    3: "FRANCE_lvl_3",
                    4: "FRANCE_lvl_4"}

cluster_level = dict_cluster_levels[st_cluster_level]


def f_chroniques(cluster_level) :
    # Liste des clusters
    requete = f"""
                SELECT DISTINCT {cluster_level}
                FROM cluster_list
                """
    clusters = [item[0] for item in f_requete_sql(requete)]

    # Liste des chroniques pour 1er cluster
    requete = f"""
            SELECT code_bss
            FROM cluster_list
            WHERE {cluster_level} = {clusters[0]}
            """
    list_chroniques = [item[0] for item in f_requete_sql(requete)]

    custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
    chroniques = pd.read_csv(f"./Clustering/data/FRANCE.csv", sep = ";", parse_dates=True, index_col="date_mesure", date_parser=custom_date_parser)

    df_clusters = pd.DataFrame()
    df_clusters[clusters[0]] = chroniques[list_chroniques].mean(axis=1)

    for index, cluster in enumerate(clusters) :
        if index > 0 :
            requete = f"""
            SELECT code_bss
            FROM cluster_list
            WHERE {cluster_level} = {cluster}
                """
            list_chroniques = [item[0] for item in f_requete_sql(requete)]

            custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
            chroniques = pd.read_csv(f"./Clustering/data/FRANCE.csv", sep = ";", parse_dates=True, index_col="date_mesure", date_parser=custom_date_parser)

            df_clusters[cluster] = chroniques[list_chroniques].mean(axis=1)
    
    figure = px.line(df_clusters, color_discrete_sequence = px.colors.qualitative.Light24)
    figure.update_layout(legend_title="")
    return figure

f_chroniques(cluster_level)



Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée


## PX GO

In [ ]:
fig = go.Figure()


fig.add_trace(go.Choroplethmapbox(data_frame = litho, 
                     geojson=litho["geometry"], 
                     locations = litho.index, 
                     color=litho["DESCR"],
                     mapbox_style='carto-positron',
                     color_discrete_map=dict_colors,
                     opacity=0.5,
                     labels= {"DESCR" : "Terrain"},
                     center={"lat": 47, "lon": 2.2},
                     zoom=4.3))


fig.add_trace(go.Scattermapbox(data,
                             lat="latitude",
                             lon="longitude",
                             color="FRANCE_lvl_4",
                             size = "dummy_column_for_size",
                             size_max = 5,
                             labels={"FRANCE_lvl_4" : "Clusters"}).data)


fig.show()

# Essais Folium

In [ ]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import folium
from folium import IFrame
from geopy.geocoders import Nominatim

In [ ]:
litho.plot()

In [ ]:
m = folium.Map([47, 2], zoom_start=7)

ft = "DESCR" 
cmap = folium.colormap.linear.YlOrRd.scale(geo_merge[ft].min(), geo_merge[ft].max()) 



folium.TileLayer('OpenStreetMap').add_to(m)
folium.GeoJson(litho,
               style_function=lambda feature: {
                   'fillOpacity' : 0.5,
                   'columns' : "DESCR",
                   'key_on' : 'litho.RGB',
                   'weight' : 1, 
                   'color' : 'black'
               }).add_to(m)
m

In [ ]:
palette = sns.color_palette("Set2")
cmap = ListedColormap(colors=palette)

carte = folium.Map([47, 2], zoom_start=7)

folium.Choropleth(
    geo_data = litho.to_json(),                  #json
    name ='choropleth',                  
    data = litho,
    key_on='OBJECTID',
    columns = ['gml_id', 'DESCR'], #columns to work on
    fill_color = 'YlGnBu',     #I passed colors Yellow,Green,Blue
    fill_opacity = 0.5,
    line_opacity = 0.2,
    nan_fill_color="purple",
    legend_name = "LEGENDE"
).add_to(carte)
carte

In [ ]:
litho.columns

In [ ]:
import folium
new_york = folium.Map(location=[40.7128,-73.9352])
new_york

In [ ]:
folium.Map(
    location=[40.7128,-73.9352],
    tiles='Stamen Toner',
    zoom_start=13
)

In [ ]:
folium.Map?